In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
# from sgt import SGT
# import pandarallel

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential

In [2]:
path = "../data/dataset.json"

with open(path) as f:
    data = json.load(f)

df = pd.json_normalize(data)
df.head()

,browser,os,locale,user_id,gender,location,sites,time,date
0,Chrome,Windows 8,de-DE,164,m,Canada/Toronto,"[{'site': 'lenta.ru', 'length': 296}, {'site':...",03:57:00,2016-08-14
1,Chrome,Windows 10,pt-PT,99,f,Netherlands/Amsterdam,"[{'site': 'windowsupdate.com', 'length': 56}, ...",13:52:00,2016-05-31
2,Chrome,Windows 10,bg-BG,28,m,Brazil/Rio de Janeiro,"[{'site': 'slack.com', 'length': 158}, {'site'...",17:40:00,2019-04-15
3,Internet Explorer,Ubuntu,en-US,163,f,New Zealand/Auckland,"[{'site': 'amazon.com', 'length': 168}, {'site...",20:23:00,2018-03-25
4,Firefox,Debian,ro-RO,177,f,New Zealand/Auckland,"[{'site': 'vk.com', 'length': 43}, {'site': 'm...",01:10:00,2018-09-08


In [5]:
# model = keras.Sequential()
# # Add an Embedding layer expecting input vocab of size 1000, and
# # output embedding dimension of size 64.
# model.add(layers.Embedding(input_dim=1000, output_dim=64))

# # Add a LSTM layer with 128 internal units.
# model.add(layers.LSTM(128))

# # Add a Dense layer with 10 units.
# model.add(layers.Dense(10))

# model.summary()

In [6]:
def seq_extract(x):
    output = []
    
    for site in x:
        output.append(site['site'].replace(".", "_"))
    return " ".join(output)

In [7]:
sites = df.sites

In [8]:
sequences = sites.apply(seq_extract).reset_index().rename(columns = {'index':'id', 'sites':'sequence'})
sequences.head()

,id,sequence
0,0,lenta_ru lenta_ru vk_com lenta_ru wikipedia_or...
1,1,windowsupdate_com amazon_com live_com akamaied...
2,2,slack_com slack_com mail_google_com vk_com fac...
3,3,amazon_com microsoft_com gvt2_com bing_net fbc...
4,4,vk_com mail_google_com lenta_ru slack_com airb...


In [12]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

In [15]:
le = LabelEncoder()
X = le.fit_transform(sequences.sequence)

In [28]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer

In [42]:
tok = Tokenizer()

In [54]:
seqs = sequences.sequence.tolist()

In [58]:
tok.fit_on_texts(seqs)

In [59]:
X = tok.texts_to_sequences(seqs)

In [64]:
df["target"] = 1*(df.user_id == 0)

In [65]:
y = df.target

In [47]:
from keras.preprocessing.sequence import pad_sequences

In [48]:
embedding_length=25
max_length=15


In [62]:
X=pad_sequences(X,truncating='post',padding='post',maxlen=max_length)

In [63]:
X[2]

array([13,  1, 13,  1, 11,  6,  1,  7,  1,  8,  1, 32,  1, 29,  1],
      dtype=int32)

In [ ]:
tok

In [86]:

model = Sequential([
                    Embedding(input_dim=len(tok.word_index)+1, 
                              output_dim=embedding_length,  input_length=15),
                    LSTM(10),
                    Dense(2, activation="softmax")
                ])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 15, 25)            3587650   
                                                                 
 lstm_4 (LSTM)               (None, 10)                1440      
                                                                 
 dense_4 (Dense)             (None, 2)                 22        
                                                                 
Total params: 3,589,112
Trainable params: 3,589,112
Non-trainable params: 0
_________________________________________________________________


In [87]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

In [88]:
model.fit(X, y, batch_size=128, epochs=20, validation_split=.2)

Epoch 1/20


ValueError: in user code:

    File "/opt/miniconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/opt/miniconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/miniconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/opt/miniconda3/lib/python3.9/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/miniconda3/lib/python3.9/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/opt/miniconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    Fi

In [71]:
embs = model.layers[0]

In [76]:
embs.embeddings.shape

TensorShape([143506, 25])